In [ ]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import os

# 브라우저 옵션 설정
options = Options()
service = Service()
options.add_argument("--start-maximized")
options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행
options.add_argument("--disable-gpu")  # GPU 비활성화 (headless 모드에서 권장)

driver = webdriver.Chrome(service=service, options=options)

# 베이스 URL 설정
base_url = "https://newslibrary.naver.com/search/searchByKeyword.naver"

# 검색어 설정
search_queries = ["검색어어"]

# 모든 기사 정보를 담을 리스트
all_articles = []
# 중복 체크를 위한 URL 집합
seen_urls = set()

for search_query in search_queries:
    page_num = 1
    while True:
        print(f"검색어 '{search_query}'의 {page_num}페이지 검색 중...")

        # 페이지 URL 설정
        url = f"https://newslibrary.naver.com/search/searchByKeyword.naver#%7B%22mode%22%3A1%2C%22sort%22%3A0%2C%22trans%22%3A%221%22%2C%22pageSize%22%3A10%2C%22keyword%22%3A%22{search_query}%22%2C%22status%22%3A%22success%22%2C%22startIndex%22%3A1%2C%22page%22%3A{page_num}%2C%22startDate%22%3A%221955-01-01%22%2C%22endDate%22%3A%221999-12-31%22%7D"
        
        # 사이트로 이동
        driver.get(url)

        # 페이지 로딩을 위한 대기 시간
        time.sleep(2)

        # 페이지 소스 가져오기
        page_source = driver.page_source

        # BeautifulSoup를 사용하여 HTML 파싱
        soup = BeautifulSoup(page_source, "html.parser")

        # 기사 목록 가져오기
        articles = soup.find_all("li")

        # 기사 정보가 없으면 종료
        if not articles:
            break

        # 기사 정보 추출
        for article in articles:
            try:
                date = article.find("li", class_="date").text.strip()
                media_name = article.find("li", class_="first").text.strip()
                page = article.find("li").find_next_sibling("li").find_next_sibling("li").text.strip()
                article_type = article.find("li").find_next_sibling("li").find_next_sibling("li").find_next_sibling("li").text.strip()
                section = article.find("li").find_next_sibling("li").find_next_sibling("li").find_next_sibling("li").find_next_sibling("li").text.strip()
                title_tag = article.find("h3").find("a")
                title = title_tag.text.strip()
                date = article.find("li", class_="date").text.strip()
                url = title_tag.get('href')
                full_url = f"https://newslibrary.naver.com{url}"
                
                # articleId 추출
                article_id_match = re.search(r'articleId=(\d{19})', full_url)
                article_id = article_id_match.group(1) if article_id_match else None

                # 중복 기사 제외
                if full_url not in seen_urls:
                    all_articles.append({
                        "media": media_name,
                        "page": page,
                        "section": section,
                        "article_type": article_type,
                        "title": title,
                        "date": date,
                        "url": full_url,
                        "articleId": article_id,
                        "search_query": search_query,
                        'author': '',
                        'content': '' 
                    })
                    seen_urls.add(full_url)
            except AttributeError:
                # 필요한 정보가 누락된 경우
                continue

        try:
            # 다음 페이지 버튼 확인 및 클릭
            if page_num % 10 == 0:
                next_button = driver.find_element(By.CLASS_NAME, 'next')
                next_button.click()
                page_num += 1
                time.sleep(2)  # 페이지 로드 대기
            else:
                next_buttons = driver.find_elements(By.CSS_SELECTOR, "div#paginate a")
                next_button_clicked = False
                for button in next_buttons:
                    if button.text == str(page_num + 1):
                        button.click()
                        page_num += 1
                        time.sleep(2)  # 페이지 로드 대기
                        next_button_clicked = True
                        break
                if not next_button_clicked:
                    print(f"더 이상 다음 페이지가 없습니다. 종료합니다. (페이지 {page_num})")
                    break
        except (ElementNotInteractableException, NoSuchElementException):
            print(f"페이지 {page_num}에서 다음 페이지로 넘어갈 수 없습니다. 종료합니다.")
            break

# 드라이버 종료
driver.quit()

# 결과 출력
for article in all_articles:
    print(f"검색어: {article['search_query']}, 매체명: {article['media']}, 지면: {article['page']}, 섹션: {article['section']}, 글 종류: {article['article_type']}, 제목: {article['title']}, 날짜: {article['date']}, 주소: {article['url']}, 아티클ID: {article['articleId']}")

# 데이터프레임으로 변환
df = pd.DataFrame(all_articles)

# 엑셀 파일 저장 경로 설정
output_file = os.path.join(os.getcwd(), "네이버 뉴스 라이브러리 기사 목록록.xlsx")

# 엑셀 파일로 저장
df.to_excel(output_file, index=False)

print(f"기사 정보가 {output_file} 파일로 저장되었습니다.")


In [ ]:
#본문 추출 방법 아티클 아이디와 기사 주소가 필요함

import requests
import pandas as pd
import os
import time

# 기본 설정
base_path = rf"경로"
file_name = "목록 파일명"

# 파일 경로 설정
input_file = os.path.join(base_path, f'{file_name}.xlsx')

# Excel 파일 읽기
df = pd.read_excel(input_file, engine='openpyxl')

# "본문" 컬럼 추가
df["본문"] = ""

# articleId 컬럼을 문자열로 변환
df['articleId'] = df['articleId'].astype(str)

# API 요청을 위한 설정
url = "https://newslibrary.naver.com/api/article/detail/json"
headers = {
    "accept": "*/*",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "priority": "u=1, i",
    "sec-ch-ua": "\"Google Chrome\";v=\"125\", \"Chromium\";v=\"125\", \"Not.A/Brand\";v=\"24\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "referer": ""
}

# 각 행에 대해 API 요청 수행 및 응답 저장
for index, row in df.iterrows():
    article_id = row['articleId']
    article_url = row['url']
    
    data = {
        "articleId": article_id,
        "detailCode": "1001000001000000000001101100000000000000000",
        "urlKey": "articleDetail",
        "viewID": "app_articleDetail",
        "requestID": "4",
        "target": "viewer"
    }
    
    # referrer 설정
    headers["referer"] = article_url

    # POST 요청
    response = requests.post(url, headers=headers, data=data)

    # 응답 확인 및 본문 추출
    if response.status_code == 200:
        text = response.text
        df.at[index, "본문"] = text
        #print(f"기사 ID: {article_id}, URL: {article_url} 처리 성공")
    else:
        print(f"기사 ID: {article_id}, URL: {article_url}에 대한 요청 실패, 상태 코드: {response.status_code}")
        print(f"응답: {response.text}")

    # 일정 시간 대기 (서버 부하 방지)
    #time.sleep(1)

# 엑셀 파일에 저장하면서 "articleId" 컬럼의 셀 서식을 텍스트로 지정
output_file_path = os.path.join(base_path, f'{file_name} 기사.xlsx')
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Sheet1')
    
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # articleId 열을 텍스트 형식으로 설정
    text_format = workbook.add_format({'num_format': '@'})
    worksheet.set_column('G:G', None, text_format)

print(f"결과가 {output_file_path}에 저장되었습니다.")
